
UCSanDiegoX: DSE200x Python for Data Science

# Final Project: Predictors of Attitudes towards Opioids (WORKING)

### Objective: To identify clusters of demographic and experiential variables that are associated with attitudes towards opioids.

### Method: Combine data from mutiple sources to create a variable identifying each survey response geography as urban, suburban, or rural. Conduct k-modes cluster analysis to determine whether there are clusters of demographic and experiential variables associated with attitudes towards opioids, including urban, suburban or rural geography.

Datasources:
1. Proprietary survey, n = 1,200
    
    - Geographic: _VGeoRegion_ (String): State; _Vpostal_ (String): Zip code
    - Demographic/Experiential: var9, var11, var12, var13, educrec, incomerec, var16, politrec, var217
    - Attitudinal:  var230, var231, var232, var233, var234, var235, var236, var237rec, var238rec, var239rec, var240rec, var241rec, var242rec, var243rec, var244rec, var245rec, var246, var247rec var248rec, var249rec, var250rec, var251rec, var252rec, var253rec, var254rec, var255rec
    

2. 2015 ZIP Code Tabulation Areas Gazetteer File, n = 33,144 
https://www.census.gov/geo/maps-data/data/gazetteer2015.html

    - _GEOID_: Five digit ZIP Code Tabulation Area Census Code; *ALAND_SQMI*: Land Area (square miles)
    

3. B25001 HOUSING UNITS by ZCTA, 2012-2016 American Community Survey 5-Year Estimates, n = 33,120 https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_16_5YR_B25001&prodType=table

    - _GEO.id2_: Five digit ZIP Code Tabulation Area Census Code; *HD01_VD01*: Housing Units, Estimated


### Identify each Zip Code Tabulation Area as Urban, Suburban, or Rural based on housing density

ZCTA housing density = Number of housing units in ZCTA/Square miles in ZCTA

Values:
0. Urban: >2,213 households per square mile; 
1. Suburban: 102 to 2,213 households per square mile; 
2. Rural: <102 households per square mile

Source: https://fivethirtyeight.com/features/how-suburban-are-big-american-cities/

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in Opioids survey dataset
df_opioids = pd.read_csv('OpioidsWKNG.csv')

In [3]:
df_opioids.head()

,Vrid,Vdatesub,Vstatus,Vcid,Vcomment,Vlanguage,Vreferer,Vsessionid,Vuseragent,Vip,...,var245rec,var247rec,var248rec,var249rec,var250rec,var251rec,var252rec,var253rec,var254rec,var255rec
0,17,3/20/2018,Complete,,,English,,1521528719_5ab0af8fe318c0.83350522,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,47.40.144.98,...,2,2,2,1,1,2,1,1,1,1
1,18,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/9ed49688-a2...,1521528831_5ab0afff9dad82.74757954,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,24.99.168.150,...,1,1,2,1,2,1,1,2,1,1
2,22,3/20/2018,Complete,,,English,,1521528941_5ab0b06d95d276.07495051,Mozilla/5.0 (Linux; Android 7.0; Moto G (4) Bu...,47.151.21.204,...,1,2,1,1,1,1,1,2,2,2
3,23,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/ed2140dc-95...,1521528964_5ab0b084821f35.43447059,Mozilla/5.0 (X11; CrOS x86_64 8872.73.0) Apple...,98.200.10.6,...,1,2,1,1,2,1,1,1,2,2
4,24,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/0529624f-1a...,1521528989_5ab0b09d6dc659.59506533,Mozilla/5.0 (iPhone; CPU iPhone OS 8_4 like Ma...,174.210.7.12,...,1,2,1,1,2,1,1,2,2,2


In [58]:
# Check zip code frequencies
df_opioids.Vpostal.value_counts()

10006    18
33132    10
20019     7
80238     6
33074     4
77450     4
20011     4
80206     4
75211     4
80203     4
90009     4
02740     4
80209     4
98103     4
90028     4
33018     4
60628     3
77072     3
76028     3
80220     3
80634     3
01824     3
22314     3
33012     3
80014     3
30518     3
10011     3
30044     3
80205     3
89147     3
         ..
30096     1
80110     1
92880     1
60640     1
92870     1
23060     1
60016     1
01835     1
90201     1
30013     1
90807     1
35603     1
98391     1
20646     1
45102     1
91605     1
91755     1
60450     1
30024     1
33312     1
98007     1
21755     1
10036     1
33316     1
40336     1
80260     1
41041     1
22152     1
92821     1
01453     1
Name: Vpostal, Length: 829, dtype: int64

In [5]:
# Display rows with one or more null values
df_opioids[df_opioids.isnull().any(axis=1)]

,Vrid,Vdatesub,Vstatus,Vcid,Vcomment,Vlanguage,Vreferer,Vsessionid,Vuseragent,Vip,...,var245rec,var247rec,var248rec,var249rec,var250rec,var251rec,var252rec,var253rec,var254rec,var255rec


In [6]:
# Replace missing values ' ' in Vpostal with missing values np.nan objects
df_opioids['Vpostal'].replace(' ', np.nan, inplace=True)

In [7]:
# Display rows with one or more null values
df_opioids[df_opioids.isnull().any(axis=1)]

,Vrid,Vdatesub,Vstatus,Vcid,Vcomment,Vlanguage,Vreferer,Vsessionid,Vuseragent,Vip,...,var245rec,var247rec,var248rec,var249rec,var250rec,var251rec,var252rec,var253rec,var254rec,var255rec
39,86,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/29a88dcd-59...,1521530117_5ab0b505f37071.67601545,Mozilla/5.0 (Linux; Android 6.0; LG-K373 Build...,107.77.173.12,...,1,1,1,1,2,2,2,1,1,1
64,125,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/faaf7464-18...,1521530549_5ab0b6b5a801f2.23483168,Mozilla/5.0 (Linux; Android 7.0; SM-G930V Buil...,72.168.144.250,...,1,2,1,1,2,1,1,2,2,2
66,127,3/20/2018,Complete,,,English,,1521530598_5ab0b6e6aa7d71.97758483,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,174.32.160.234,...,1,1,2,2,1,1,2,1,1,1
113,206,3/20/2018,Complete,,,English,,1521532036_5ab0bc843e3185.48398254,Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like M...,174.255.133.179,...,2,2,1,2,1,1,1,2,2,1
161,284,3/20/2018,Complete,,,English,,1521557945_5ab121b9cfd0a4.06084619,Mozilla/5.0 (Linux; Android 6.0; BLU STUDIO G2...,38.132.117.101,...,2,2,1,2,2,1,2,2,2,2
164,292,3/20/2018,Complete,,,English,,1521558007_5ab121f7b8d0d2.49278047,Mozilla/5.0 (Linux; Android 6.0; BLU ADVANCE 4...,38.132.117.108,...,1,1,1,1,1,1,1,1,1,1
167,297,3/20/2018,Complete,,,English,,1521558148_5ab12284dea6e0.56965142,Mozilla/5.0 (Windows NT 6.3) AppleWebKit/537.3...,172.164.1.183,...,2,1,1,1,1,1,1,1,1,1
172,304,3/20/2018,Complete,,,English,,1521558222_5ab122ce216622.35270772,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,172.164.17.238,...,1,1,1,1,1,1,1,1,1,1
187,321,3/20/2018,Complete,,,English,,1521558472_5ab123c824b908.35655993,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,172.164.17.91,...,1,1,1,1,1,1,1,1,1,1
194,331,3/20/2018,Complete,,,English,,1521558333_5ab1233d2033f3.78203647,Mozilla/5.0 (Linux; Android 4.4.2; DASH C MUSI...,38.132.117.107,...,1,1,1,1,1,1,1,1,1,1


In [8]:
# Drop rows with missing values in Vpostal
df2_opioids=df_opioids.dropna(subset=['Vpostal'])

In [9]:
# Check number of rows before and after dropna
before_rows = df_opioids.shape[0]
print(before_rows)

1200


In [10]:
after_rows = df2_opioids.shape[0]
print(after_rows)

1101


In [11]:
# Check how many of the remaining records have states attached
df2_opioids.VGeoRegion.value_counts()

TX    168
CA    151
GA     90
WA     87
MA     87
CO     86
IL     81
NY     80
FL     68
VA     39
MD     32
DC     26
NJ     18
MI     10
OH      8
NC      7
KY      5
PA      5
AZ      5
CT      4
WI      4
IN      4
NV      4
OR      3
ID      3
ME      3
HI      2
SC      2
AL      2
RI      2
DE      2
UT      2
KS      1
WV      1
AK      1
AR      1
MS      1
IA      1
WY      1
MN      1
LA      1
OK      1
MO      1
Name: VGeoRegion, dtype: int64

In [12]:
df2_opioids['Vpostal']

0       49048
1       30022
2       92683
3       77036
4       90026
5       60073
6       98501
7       77040
8       10463
9       90013
10      95124
11      76033
12      10016
13      01463
14      75134
15      90004
16      75115
17      33025
18      94127
19      75270
20      92126
21      97403
22      91741
23      92415
24      30039
25      90038
26      60438
27      95132
28      92882
29      98405
        ...  
1167    20011
1168    20814
1169    27804
1170    20003
1171    20170
1172    22310
1173    22554
1174    33837
1175    84604
1176    20004
1177    20874
1178    20110
1179    21223
1180    20171
1181    20175
1182    20191
1183    20003
1184    22310
1185    83001
1186    20871
1187    79845
1188    20176
1189    20170
1190    22060
1192    11768
1193    30152
1195    33065
1197    30032
1198    44313
1199    72712
Name: Vpostal, Length: 1101, dtype: object

In [54]:
# Check how many unique zip codes remain
df2_opioids['Vpostal'].nunique()

829

In [13]:
# Create new GEOID variable by transforming Vpostal into an integer (stripping the leading zeros from Vpostal)
strip_Vpostal = df2_opioids['Vpostal'].apply(int)

In [14]:
strip_Vpostal.head()

0    49048
1    30022
2    92683
3    77036
4    90026
Name: Vpostal, dtype: int64

In [15]:
len(strip_Vpostal)

1101

In [16]:
# Check zip codes count
strip_Vpostal.value_counts()

10006    18
33132    10
20019     7
80238     6
90028     4
80209     4
90009     4
33018     4
98103     4
80203     4
2740      4
75211     4
20011     4
33074     4
80206     4
77450     4
14009     3
2301      3
77072     3
80014     3
77494     3
22314     3
77503     3
94541     3
77063     3
30044     3
76028     3
89147     3
1902      3
23220     3
         ..
77095     1
77092     1
11554     1
75040     1
77084     1
77081     1
46360     1
30022     1
75081     1
75083     1
30046     1
98375     1
30066     1
77026     1
93550     1
75115     1
30052     1
40291     1
98104     1
30045     1
21223     1
30043     1
19802     1
30039     1
30038     1
75093     1
30034     1
60007     1
30032     1
45056     1
Name: Vpostal, Length: 829, dtype: int64

In [17]:
df3_opioids = df2_opioids.assign(GEOID = strip_Vpostal)

In [18]:
df3_opioids.head()

,Vrid,Vdatesub,Vstatus,Vcid,Vcomment,Vlanguage,Vreferer,Vsessionid,Vuseragent,Vip,...,var247rec,var248rec,var249rec,var250rec,var251rec,var252rec,var253rec,var254rec,var255rec,GEOID
0,17,3/20/2018,Complete,,,English,,1521528719_5ab0af8fe318c0.83350522,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,47.40.144.98,...,2,2,1,1,2,1,1,1,1,49048
1,18,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/9ed49688-a2...,1521528831_5ab0afff9dad82.74757954,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,24.99.168.150,...,1,2,1,2,1,1,2,1,1,30022
2,22,3/20/2018,Complete,,,English,,1521528941_5ab0b06d95d276.07495051,Mozilla/5.0 (Linux; Android 7.0; Moto G (4) Bu...,47.151.21.204,...,2,1,1,1,1,1,2,2,2,92683
3,23,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/ed2140dc-95...,1521528964_5ab0b084821f35.43447059,Mozilla/5.0 (X11; CrOS x86_64 8872.73.0) Apple...,98.200.10.6,...,2,1,1,2,1,1,1,2,2,77036
4,24,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/0529624f-1a...,1521528989_5ab0b09d6dc659.59506533,Mozilla/5.0 (iPhone; CPU iPhone OS 8_4 like Ma...,174.210.7.12,...,2,1,1,2,1,1,2,2,2,90026


In [19]:
# Check new column GEOID
GEOIDslice = ['Vrid', 'Vpostal','GEOID']
GEOIDcheck = df3_opioids[GEOIDslice]
print(GEOIDcheck)

      Vrid Vpostal  GEOID
0       17   49048  49048
1       18   30022  30022
2       22   92683  92683
3       23   77036  77036
4       24   90026  90026
5       25   60073  60073
6       28   98501  98501
7       30   77040  77040
8       35   10463  10463
9       37   90013  90013
10      38   95124  95124
11      40   76033  76033
12      41   10016  10016
13      42   01463   1463
14      48   75134  75134
15      50   90004  90004
16      51   75115  75115
17      52   33025  33025
18      53   94127  94127
19      54   75270  75270
20      55   92126  92126
21      59   97403  97403
22      61   91741  91741
23      62   92415  92415
24      63   30039  30039
25      65   90038  90038
26      66   60438  60438
27      69   95132  95132
28      71   92882  92882
29      72   98405  98405
...    ...     ...    ...
1167  3098   20011  20011
1168  3102   20814  20814
1169  3121   27804  27804
1170  3125   20003  20003
1171  3129   20170  20170
1172  3173   22310  22310
1173  3184  

In [20]:
# Save records with zip codes as a new csv file
df3_opioids.to_csv("Opioids_ZCSubset.csv", index=False)

In [21]:
# Read in ZIP Code Tabulation Areas Gazetteer File
df_ZCTA = pd.read_csv("2015_Gaz_zcta_national.txt", sep = "\t")
df_ZCTA.head()

,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,601,166659883,799293,64.348,0.309,18.180555,-66.749961
1,602,79287203,4448761,30.613,1.718,18.361945,-67.175597
2,603,81884524,184089,31.616,0.071,18.455183,-67.119887
3,606,109579998,12487,42.309,0.005,18.158345,-66.932911
4,610,93021290,4171994,35.916,1.611,18.295366,-67.125135


In [22]:
len(df_ZCTA.index)

33144

In [23]:
# Create dataset with GEOID and ALAND_SQMI only, save as .csv
features = ['GEOID', 'ALAND_SQMI']
df2_ZCTA = df_ZCTA[features]

In [24]:
df2_ZCTA.head()

,GEOID,ALAND_SQMI
0,601,64.348
1,602,30.613
2,603,31.616
3,606,42.309
4,610,35.916


In [25]:
# Save land area by ZCTA as a new csv file
df2_ZCTA.to_csv("LandArea_ZCTA.csv", index=False)

In [26]:
# Read in housing unit counts by ZCTA from ACS data
df_HU = pd.read_csv("ACS_16_5YR_B25001_with_ann.csv", header=0, skiprows=[1])
df_HU.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01
0,8600000US00601,601,ZCTA5 00601,7290,154
1,8600000US00602,602,ZCTA5 00602,17312,196
2,8600000US00603,603,ZCTA5 00603,24594,334
3,8600000US00606,606,ZCTA5 00606,2763,118
4,8600000US00610,610,ZCTA5 00610,12265,170


In [27]:
# Rename GEO.id2 to GEOID
df_HU.rename(columns = {'GEO.id2':'GEOID'}, inplace=True)
df_HU.head()

,GEO.id,GEOID,GEO.display-label,HD01_VD01,HD02_VD01
0,8600000US00601,601,ZCTA5 00601,7290,154
1,8600000US00602,602,ZCTA5 00602,17312,196
2,8600000US00603,603,ZCTA5 00603,24594,334
3,8600000US00606,606,ZCTA5 00606,2763,118
4,8600000US00610,610,ZCTA5 00610,12265,170


In [28]:
# Create dataset with GEOID and HD01_VD01 only, save as .csv
features2 = ['GEOID', 'HD01_VD01']
df2_HU= df_HU[features2]

In [29]:
df2_HU.head()

,GEOID,HD01_VD01
0,601,7290
1,602,17312
2,603,24594
3,606,2763
4,610,12265


In [30]:
len(df2_HU)

33120

In [32]:
# Save housing units by ZCTA as a new csv file
df2_HU.to_csv("HousingUnits_ZCTA.csv", index=False)

In [35]:
# Add housing units variable 'HD01_VD01' from df2_HU to survey dataframe df3_opioids by merging on GEOID
df_merged = pd.merge(df3_opioids, df2_HU, how='left', on='GEOID')
df_merged.head()

,Vrid,Vdatesub,Vstatus,Vcid,Vcomment,Vlanguage,Vreferer,Vsessionid,Vuseragent,Vip,...,var248rec,var249rec,var250rec,var251rec,var252rec,var253rec,var254rec,var255rec,GEOID,HD01_VD01
0,17,3/20/2018,Complete,,,English,,1521528719_5ab0af8fe318c0.83350522,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,47.40.144.98,...,2,1,1,2,1,1,1,1,49048,10782.0
1,18,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/9ed49688-a2...,1521528831_5ab0afff9dad82.74757954,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,24.99.168.150,...,2,1,2,1,1,2,1,1,30022,24854.0
2,22,3/20/2018,Complete,,,English,,1521528941_5ab0b06d95d276.07495051,Mozilla/5.0 (Linux; Android 7.0; Moto G (4) Bu...,47.151.21.204,...,1,1,1,1,1,2,2,2,92683,28291.0
3,23,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/ed2140dc-95...,1521528964_5ab0b084821f35.43447059,Mozilla/5.0 (X11; CrOS x86_64 8872.73.0) Apple...,98.200.10.6,...,1,1,2,1,1,1,2,2,77036,30892.0
4,24,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/0529624f-1a...,1521528989_5ab0b09d6dc659.59506533,Mozilla/5.0 (iPhone; CPU iPhone OS 8_4 like Ma...,174.210.7.12,...,1,1,2,1,1,2,2,2,90026,26958.0


In [37]:
# Add land area variable 'ALAND_SQMI' from df2_ZCTA to merged dataframe df_merged by merging on GEOID
df_merged = pd.merge(df_merged, df2_ZCTA, how='left', on='GEOID')
df_merged.head()

,Vrid,Vdatesub,Vstatus,Vcid,Vcomment,Vlanguage,Vreferer,Vsessionid,Vuseragent,Vip,...,var249rec,var250rec,var251rec,var252rec,var253rec,var254rec,var255rec,GEOID,HD01_VD01,ALAND_SQMI
0,17,3/20/2018,Complete,,,English,,1521528719_5ab0af8fe318c0.83350522,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,47.40.144.98,...,1,1,2,1,1,1,1,49048,10782.0,37.908
1,18,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/9ed49688-a2...,1521528831_5ab0afff9dad82.74757954,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,24.99.168.150,...,1,2,1,1,2,1,1,30022,24854.0,25.444
2,22,3/20/2018,Complete,,,English,,1521528941_5ab0b06d95d276.07495051,Mozilla/5.0 (Linux; Android 7.0; Moto G (4) Bu...,47.151.21.204,...,1,1,1,1,2,2,2,92683,28291.0,9.993
3,23,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/ed2140dc-95...,1521528964_5ab0b084821f35.43447059,Mozilla/5.0 (X11; CrOS x86_64 8872.73.0) Apple...,98.200.10.6,...,1,2,1,1,1,2,2,77036,30892.0,7.155
4,24,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/0529624f-1a...,1521528989_5ab0b09d6dc659.59506533,Mozilla/5.0 (iPhone; CPU iPhone OS 8_4 like Ma...,174.210.7.12,...,1,2,1,1,2,2,2,90026,26958.0,4.224


In [38]:
# Create new variable 'HD' housing density (housing units per square mile = HD01_VD01/ALAND_SQMI)
df_merged['HD'] = df_merged['HD01_VD01']/df_merged['ALAND_SQMI']
df_merged.head()

,Vrid,Vdatesub,Vstatus,Vcid,Vcomment,Vlanguage,Vreferer,Vsessionid,Vuseragent,Vip,...,var250rec,var251rec,var252rec,var253rec,var254rec,var255rec,GEOID,HD01_VD01,ALAND_SQMI,HD
0,17,3/20/2018,Complete,,,English,,1521528719_5ab0af8fe318c0.83350522,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,47.40.144.98,...,1,2,1,1,1,1,49048,10782.0,37.908,284.425451
1,18,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/9ed49688-a2...,1521528831_5ab0afff9dad82.74757954,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,24.99.168.150,...,2,1,1,2,1,1,30022,24854.0,25.444,976.811822
2,22,3/20/2018,Complete,,,English,,1521528941_5ab0b06d95d276.07495051,Mozilla/5.0 (Linux; Android 7.0; Moto G (4) Bu...,47.151.21.204,...,1,1,1,2,2,2,92683,28291.0,9.993,2831.081757
3,23,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/ed2140dc-95...,1521528964_5ab0b084821f35.43447059,Mozilla/5.0 (X11; CrOS x86_64 8872.73.0) Apple...,98.200.10.6,...,2,1,1,1,2,2,77036,30892.0,7.155,4317.540182
4,24,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/0529624f-1a...,1521528989_5ab0b09d6dc659.59506533,Mozilla/5.0 (iPhone; CPU iPhone OS 8_4 like Ma...,174.210.7.12,...,2,1,1,2,2,2,90026,26958.0,4.224,6382.102273


In [52]:
# Check number of values in HD
len(df_merged['HD'])

1074

In [41]:
# Display rows with one or more null values
df_merged[df_merged.isnull().any(axis=1)]

,Vrid,Vdatesub,Vstatus,Vcid,Vcomment,Vlanguage,Vreferer,Vsessionid,Vuseragent,Vip,...,var250rec,var251rec,var252rec,var253rec,var254rec,var255rec,GEOID,HD01_VD01,ALAND_SQMI,HD
23,62,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/7a22080b-ef...,1521529681_5ab0b35183caa6.27000677,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,4.7.99.141,...,2,2,2,1,1,1,92415,NaN,NaN,NaN
77,151,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/d22ee76d-6a...,1521530904_5ab0b8185aa897.38419568,Mozilla/5.0 (Linux; Android 5.1.1; SM-G530T Bu...,172.58.95.123,...,2,1,1,2,2,2,94165,NaN,NaN,NaN
103,194,3/20/2018,Complete,,,English,,1521531684_5ab0bb248337f7.04350648,Mozilla/5.0 (Linux; Android 6.0.1; SM-J327P Bu...,66.87.139.93,...,2,1,1,2,1,2,98464,NaN,NaN,NaN
133,242,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/492a753a-a6...,1521532873_5ab0bfc9e22790.07401685,Mozilla/5.0 (Linux; Android 7.0; SAMSUNG SM-J3...,172.56.34.176,...,1,1,1,2,2,2,11249,NaN,NaN,NaN
168,309,3/20/2018,Complete,,,English,,1521558337_5ab12341816f12.84711703,Mozilla/5.0 (Linux; Android 6.0; BLU ADVANCE 4...,38.132.110.34,...,1,1,1,2,1,1,33074,NaN,NaN,NaN
207,370,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/333a6364-23...,1521559491_5ab127c36c1b04.99250915,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,209.208.213.52,...,1,1,1,1,1,1,33148,NaN,NaN,NaN
222,403,3/20/2018,Complete,,,English,,1521560007_5ab129c7cebf25.58288703,Mozilla/5.0 (Linux; Android 6.0; BLU ADVANCE 4...,38.132.110.34,...,1,1,1,1,1,1,33074,NaN,NaN,NaN
236,429,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/3400f312-29...,1521560609_5ab12c21cb0899.40876162,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,198.134.2.62,...,2,1,1,2,1,1,30301,NaN,NaN,NaN
241,438,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/b3a5533e-75...,1521560744_5ab12ca8ae8f01.85334469,Mozilla/5.0 (Linux; Android 7.0; SAMSUNG-SM-G9...,129.110.241.80,...,1,1,1,1,1,1,75083,NaN,NaN,NaN
247,453,3/20/2018,Complete,,,English,,1521558732_5ab124cc856289.34135799,Mozilla/5.0 (Linux; Android 6.0; BLU DASH L2 B...,38.132.110.35,...,2,1,1,2,1,2,33074,NaN,NaN,NaN


In [42]:
before_rows = df_merged.shape[0]
print(before_rows)

1101


In [43]:
# Drop rows with missing values
df_merged = df_merged.dropna()

In [45]:
after_rows = df_merged.shape[0]
print(after_rows)

1074


In [46]:
# Check number of rows dropped
before_rows - after_rows

27

In [55]:
# Check how many unique zip codes remain
df_merged['GEOID'].nunique()

810

In [57]:
# Create new variable 'CT' community type (Urban: >2,213 hu/sqmi; Suburban: 102 to 2,213 hu/sqmi; Rural: <102 hu/sqmi)
df_merged['CT']=np.where(df_merged['HD']>2213, 0,(np.where(df_merged['HD']<102,2,1)))
df_merged.head()

,Vrid,Vdatesub,Vstatus,Vcid,Vcomment,Vlanguage,Vreferer,Vsessionid,Vuseragent,Vip,...,var251rec,var252rec,var253rec,var254rec,var255rec,GEOID,HD01_VD01,ALAND_SQMI,HD,CT
0,17,3/20/2018,Complete,,,English,,1521528719_5ab0af8fe318c0.83350522,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,47.40.144.98,...,2,1,1,1,1,49048,10782.0,37.908,284.425451,1
1,18,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/9ed49688-a2...,1521528831_5ab0afff9dad82.74757954,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,24.99.168.150,...,1,1,2,1,1,30022,24854.0,25.444,976.811822,1
2,22,3/20/2018,Complete,,,English,,1521528941_5ab0b06d95d276.07495051,Mozilla/5.0 (Linux; Android 7.0; Moto G (4) Bu...,47.151.21.204,...,1,1,2,2,2,92683,28291.0,9.993,2831.081757,0
3,23,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/ed2140dc-95...,1521528964_5ab0b084821f35.43447059,Mozilla/5.0 (X11; CrOS x86_64 8872.73.0) Apple...,98.200.10.6,...,1,1,1,2,2,77036,30892.0,7.155,4317.540182,0
4,24,3/20/2018,Complete,,,English,https://s.cint.com/Consent/Collect/0529624f-1a...,1521528989_5ab0b09d6dc659.59506533,Mozilla/5.0 (iPhone; CPU iPhone OS 8_4 like Ma...,174.210.7.12,...,1,1,2,2,2,90026,26958.0,4.224,6382.102273,0


In [59]:
# Check CT frequencies
df_merged.CT.value_counts()

1    630
0    396
2     48
Name: CT, dtype: int64

In [60]:
# Save merged dataframe as a new csv file
df_merged.to_csv("OpioidsMerged.csv", index=False)

### Use cluster analysis to identify associated clusters of demographic/experiential variables and opioid attitudes

In [10]:
from kmodes.kmodes import KModes
import python_utils
from itertools import cycle, islice
import matplotlib.pyplot as plt
from pandas.tools.plotting import parallel_coordinates
%matplotlib inline

In [87]:
# Select features of interest and create dataframe for cluster analysis
features = ['GEOID','var9', 'var11', 'var12', 'var13', 'educrec', 'incomerec', 'var16', 'politrec', 'var217',
            'var230', 'var231', 'var232', 'var233', 'var234', 'var235', 'var236', 'var237rec', 'var238rec', 
            'var239rec', 'var240rec', 'var241rec', 'var242rec', 'var243rec', 'var244rec', 'var245rec', 'var246', 
            'var247rec','var248rec', 'var249rec', 'var250rec', 'var251rec', 'var252rec', 'var253rec', 'var254rec',
            'var255rec', 'CT']
cslice = df_merged[features]
cslice.head()

,GEOID,var9,var11,var12,var13,educrec,incomerec,var16,politrec,var217,...,var247rec,var248rec,var249rec,var250rec,var251rec,var252rec,var253rec,var254rec,var255rec,CT
0,49048,10001,10023,10026,10031,3,2,10052,1,10657,...,2,2,1,1,2,1,1,1,1,1
1,30022,10006,10024,10027,10031,3,3,10055,2,10657,...,1,2,1,2,1,1,2,1,1,1
2,92683,10001,10023,10025,10034,2,2,10056,3,10656,...,2,1,1,1,1,1,2,2,2,0
3,77036,10004,10024,10025,10036,2,1,10052,3,10656,...,2,1,1,2,1,1,1,2,2,0
4,90026,10001,10024,10029,10034,2,2,10056,1,10656,...,2,1,1,2,1,1,2,2,2,0


In [88]:
cslice.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
GEOID,1074.0,54169.885475,33393.072468,1331.0,22038.25,60601.5,80238.0,99504.0
var9,1074.0,10005.189944,2.994753,10001.0,10003.00,10005.0,10008.0,10010.0
var11,1074.0,10023.510242,0.500128,10023.0,10023.00,10024.0,10024.0,10024.0
var12,1074.0,10027.600559,1.524463,10025.0,10026.00,10027.0,10029.0,10030.0
var13,1074.0,10031.665736,1.216920,10031.0,10031.00,10031.0,10032.0,10036.0
educrec,1074.0,2.764432,1.009462,1.0,2.00,3.0,4.0,4.0
incomerec,1074.0,1.965549,0.782929,1.0,1.00,2.0,3.0,3.0
var16,1074.0,10053.208566,1.666310,10052.0,10052.00,10052.0,10054.0,10058.0
politrec,1074.0,2.002793,1.024850,1.0,1.00,2.0,3.0,5.0
var217,1074.0,10657.161080,1.195336,10656.0,10656.00,10657.0,10657.0,10661.0


In [89]:
# Next step: Use the elbow method to determine the optimal number of clusters for k-modes clustering